In [ ]:
# Check GPU version
!nvidia-smi

In [ ]:
# install packages
!pip install catboost
!pip install optuna
!pip install wandb

In [ ]:
# 即時監控colab資源
import wandb

# connect with jim107225017/colab/20210526
wandb.init(project='colab', entity='jim107225017', name='CPU_GPU', id='20210526')

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd

# Optuna
import optuna
from optuna.samplers import TPESampler

import catboost
from catboost import CatBoostClassifier

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from joblib import load, dump

def check_gpu_support():
    try:
        data = np.random.rand(1000, 10)
        label = np.random.randint(2, size=1000)
        train_data = lightgbm.Dataset(data, label=label)
        params = {'device': 'gpu'}
        gbm = lightgbm.train(params, train_set=train_data)
        return True
    except Exception as e:
        return False

print(check_gpu_support())

In [ ]:
# connect with Google Cloud
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/colab/TPS Jun"
# path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - Jun 2021'
os.chdir(path)

In [ ]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
cat_col = [i for i in all_data.columns if i not in ['id', 'target']]
target_col = 'target'

x = df_train[cat_col]

# Label Y
le = LabelEncoder()
y = le.fit_transform(df_train[target_col])

In [ ]:
# Optuna
SEED = 20210605
sampler = TPESampler(seed=SEED)

def objective(trial, kaggle_metrics='LogLoss', predic_proba=True):
    train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, random_state=SEED, shuffle=True, stratify=y)
    
    params = {'iterations':trial.suggest_int("iterations", 4000, 25000),
              'od_wait':trial.suggest_int('od_wait', 500, 2300),
              'loss_function':'MultiClass',
              'task_type':"GPU",
              'eval_metric':'MultiClass',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'random_seed': SEED,
              'learning_rate' : trial.suggest_uniform('learning_rate',0.005,1),
              'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              'subsample': trial.suggest_uniform('subsample',0,1),
              'random_strength': trial.suggest_uniform('random_strength',10,50),
              'depth': trial.suggest_int('depth',1,15),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
             }

    clf = CatBoostClassifier(**param)
    clf.fit(train_x, train_y,
              eval_set=[(valid_x, valid_y)],
              early_stopping_rounds=100,
              verbose=False, cat_features=cat_col)
    
    if predic_proba:
        preds = clf.predict_proba(valid_x)
        preds = np.float64(preds)
    else:
        preds = clf.predict(valid_x)
    
    if kaggle_metrics == 'LogLoss':
        result = log_loss(valid_y, preds)
    elif kaggle_metrics == 'AUC':
        result = roc_auc_score(valid_y, preds)
    elif kaggle_metrics == 'Acc':
        result = accuracy_score(valid_y, preds)
    
    return result


In [ ]:
OPTUNA_OPTIMIZATION = True

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))
    display(optuna.visualization.plot_parallel_coordinate(study))

In [ ]:
# create best model
best_param = study.best_trial.params
print(best_param)
best_model = LGBMRegressor(**best_param)

# save best model
PROJECT_NAME = 'catboost(optuna)-20210605'
dump(best_model, f'{PROJECT_NAME}.joblib')

In [ ]:
test_preds=None

kf = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 42)
for fold, (tr_index , val_index) in enumerate(kf.split(X.values , Y.values)):
    
    print("-" * 50)
    print(f"Fold {fold + 1}")
    
    x_train,x_val = X.values[tr_index] , X.values[val_index]
    y_train,y_val = Y.values[tr_index] , Y.values[val_index]
        
    eval_set = [(x_val, y_val)]
    
    model =CatBoostClassifier(**cat_params)
    model.fit(x_train, y_train, eval_set = eval_set, verbose = False)
    
    train_preds = model.predict(x_train)    
    val_preds = model.predict_proba(x_val)
    
    print(log_loss(y_val, val_preds))
    
    if test_preds is None:
        test_preds = model.predict_proba(test[cols].values)
    else:
        test_preds += model.predict_proba(test[cols].values)

print("-" * 50)
test_preds /= 10

In [ ]:
# load best model
best_model = load(f'{PROJECT_NAME}.joblib')

In [ ]:
# predict probability
x_test = df_test.drop('id', axis=1)
result = best_model.predict_proba(x_test)

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result
sub.to_csv(f'{PROJECT_NAME}.csv', index=False)